## A/B Testing on Datacamp
## 07/01/2023
---
### Table of Content
1) What is A/B Testing

**1) What is A/B Testing**  
  
* An experiment designed to test which version is better  
* **based on metric(s)** depending on the case  
* using **random assigment** & analyzing results  
       
* **Fundamental steps**  
    * Specify the goal & designs/ experiences  
    * *Randomly* sample users for enrollment  
    * *Randomly* assign users to:  
        * *control variant:* current state  
        * *treatment/test variant(s):* new design  
    * Log user actions & compute metrics  
    * Test for statistically significant differences  
    
* *Randomization* provides:
    * generalizability & representativeness
    * minimizing bias between groups
    * establishing causality by isolating treatment effect